# 🛡️ Security Events Analysis - Microsoft Sentinel Data Lake

**Analyze Windows Security Events for threat detection and security monitoring.**

## 🎯 Security Analysis Covered

| Category | Detection | Impact |
|----------|-----------|---------|
| **🔐 Authentication** | Failed logons, brute force attacks | Critical |
| **⚡ Process Activity** | Suspicious process execution | High |
| **👥 Account Management** | User/group changes | Medium |
| **🌐 Network Access** | Lateral movement patterns | High |
| **📊 Risk Assessment** | Overall security posture | Medium |

## ⚙️ Quick Setup
1. Update `PRIMARY_WORKSPACE` in the config cell below
2. Run all cells
3. Analyze the security event findings

---

## 🚀 Quick Start Guide

**Simple 3-Step Process:**
1. **Update workspace name** in the config cell below
2. **Run all cells** to get security insights  
3. **Review findings** and take action on critical alerts

**💡 What You'll Get:**
- **Authentication threats** (brute force, failed logons)
- **Process anomalies** (suspicious executions, off-hours activity)
- **Account changes** (user/group modifications)
- **Network patterns** (lateral movement indicators)
- **Risk assessment** (overall security score)

---

In [6]:
# Import libraries and setup
import pandas as pd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

from sentinel_lake.providers import MicrosoftSentinelProvider
from pyspark.sql.functions import (
    col, count, desc, asc, when, 
    countDistinct, sum as spark_sum, avg,
    hour, dayofweek, date_trunc,
    current_timestamp, expr, lit
)
import matplotlib.pyplot as plt

# Set visualization style
try:
    plt.style.use('seaborn-v0_8')
except:
    plt.style.use('default')

# Initialize data provider
data_provider = MicrosoftSentinelProvider(spark)

print("✅ Libraries imported and data provider initialized")

# 🔄 WORKSPACE CONFIGURATION
# ===========================================
# 🎯 SIMPLE SETUP: Copy the workspace names from your setup notebook!

PRIMARY_WORKSPACE = "ak-SecOps"      # 🏢 Copy your primary workspace name here
ENTRA_WORKSPACE = "ak-SecOps"        # 🔵 Activity logs are in primary workspace; use "System Tables" for asset data

# Analysis configuration
ANALYSIS_HOURS = 24                  # 📅 Hours of data to analyze (24 hours = 1 day)
SENTINEL_ENVIRONMENT = True          # 🔍 Enable advanced analysis features

# Advanced settings (optional - can leave as defaults)
WORKSPACE_MAPPING = {
    'SecurityEvent': PRIMARY_WORKSPACE,
    'SigninLogs': PRIMARY_WORKSPACE,
    'DeviceEvents': PRIMARY_WORKSPACE,
    'DeviceProcessEvents': PRIMARY_WORKSPACE,
    'DeviceNetworkEvents': PRIMARY_WORKSPACE,
    'CommonSecurityLog': PRIMARY_WORKSPACE,
    'Syslog': PRIMARY_WORKSPACE,
    'AuditLogs': PRIMARY_WORKSPACE,
    'EntraUsers': ENTRA_WORKSPACE,
    'EntraGroups': ENTRA_WORKSPACE,
    'EntraApplications': ENTRA_WORKSPACE
}

print(f"\n🎯 SECURITY EVENTS ANALYSIS CONFIGURATION:")
print(f"🏢 Primary workspace: '{PRIMARY_WORKSPACE}'")
print(f"🔵 Entra workspace: '{ENTRA_WORKSPACE}'")
print(f"? Analysis window: {ANALYSIS_HOURS} hours")

# Configuration validation
if PRIMARY_WORKSPACE == "YOUR_WORKSPACE_NAME_HERE":
    print(f"\n⚠️  CONFIGURATION NEEDED!")
    print(f"📝 Please update the workspace names above:")
    print(f"   1. Replace PRIMARY_WORKSPACE with your actual workspace name")
    print(f"   2. Replace ENTRA_WORKSPACE with your Entra workspace name")
    print(f"   3. Re-run this cell")
elif PRIMARY_WORKSPACE == "test-workspace":
    print(f"\n✅ DEMO MODE: Using test configuration")
    print(f"💡 Configuration system is working correctly!")
    print(f"? For real analysis, replace with your actual workspace names")
else:
    print(f"\n✅ Configuration looks good!")
    print(f"🛡️ Ready for security events analysis in your environment")
    
    # Show workspace mapping
    print(f"\n📊 Table-to-workspace mapping:")
    for table, workspace in WORKSPACE_MAPPING.items():
        print(f"   • {table} → {workspace}")

print(f"\n🎯 SECURITY EVENTS ANALYSIS READY!")
print(f"🔍 This notebook will analyze Windows Security Events for threat detection")

# Helper function for safe table checking using discovered mapping
def safe_table_check(table_name, workspace_name=None):
    """Safely check table availability using workspace mapping"""
    try:
        # Use workspace mapping from configuration
        if workspace_name is None:
            workspace_name = WORKSPACE_MAPPING.get(table_name, PRIMARY_WORKSPACE)
        
        df = data_provider.read_table(table_name, workspace_name)
        
        # Get basic stats with small sample for performance
        sample_count = df.limit(100).count()
        columns = df.columns
        
        return {
            'available': True,
            'sample_rows': sample_count,
            'total_columns': len(columns),
            'columns': columns[:5],  # Show first 5 columns
            'workspace': workspace_name,
            'error': None
        }
    except Exception as e:
        return {
            'available': False,
            'sample_rows': 0,
            'total_columns': 0,
            'columns': [],
            'workspace': workspace_name,
            'error': str(e)
        }

# Quick verification of SecurityEvent table
print(f"\n🔍 VERIFYING SECURITY EVENT DATA...")
print("=" * 40)

# Test security-related tables
security_tables_to_test = ["SecurityEvent", "Syslog", "CommonSecurityLog"]
accessible_security_tables = []

for table in security_tables_to_test:
    result = safe_table_check(table)
    if result['available']:
        accessible_security_tables.append(table)
        print(f"✅ {table}: {result['sample_rows']} sample rows in '{result['workspace']}'")
    else:
        print(f"❌ {table}: Not accessible ({result['error'][:50]}...)")

print(f"\n📊 SECURITY EVENT DATA AVAILABILITY:")
print(f"   ✅ Accessible tables: {len(accessible_security_tables)}/{len(security_tables_to_test)}")

if accessible_security_tables:
    print(f"   📋 Ready for analysis: {', '.join(accessible_security_tables)}")
    print(f"\n🚀 Ready to begin security events analysis!")
    
    # Set flag for analysis availability
    security_events_available = "SecurityEvent" in accessible_security_tables
    
    if security_events_available:
        # Load SecurityEvent data for analysis
        security_table_info = safe_table_check("SecurityEvent")
        security_events = data_provider.read_table("SecurityEvent", security_table_info['workspace'])
        
        # Filter to analysis window
        recent_events = security_events.filter(
            col("TimeGenerated") >= (current_timestamp() - expr(f"INTERVAL {ANALYSIS_HOURS} HOURS"))
        )
        
        recent_count = recent_events.count()
        print(f"📊 Security events in last {ANALYSIS_HOURS} hours: {recent_count:,}")
        
        if recent_count == 0:
            print("⚠️  No recent events found - consider increasing ANALYSIS_HOURS")
            
    else:
        print("⚠️ SecurityEvent table not available - analysis will be limited")
        
else:
    print(f"   ⚠️ Limited security event data available")
    print(f"   💡 This could mean:")
    print(f"      • Security event data is still being ingested")
    print(f"      • Windows Event Log collection not configured") 
    print(f"      • Permissions need adjustment")
    print(f"   📝 The analysis sections will adapt to available data")
    
    security_events_available = False

print("\n" + "="*60)

## Event Overview

Check what types of security events are available in the data.

In [7]:
# Event ID distribution and common security events
if security_events_available:
    print("📊 SECURITY EVENT OVERVIEW")
    print("=" * 25)
    
    # Get Event ID distribution
    event_distribution = recent_events.groupBy("EventID", "Activity") \
        .agg(count("*").alias("EventCount")) \
        .orderBy(desc("EventCount"))
    
    print("🏆 TOP SECURITY EVENTS (Last 24 hours):")
    event_distribution.show(10, truncate=False)
    
    # Critical security event IDs
    critical_events = {
        4624: "Successful Logon", 4625: "Failed Logon", 4634: "Logoff",
        4688: "Process Creation", 4720: "User Account Created", 4726: "User Account Deleted",
        4728: "User Added to Group", 4648: "Explicit Credentials", 5140: "Network Share Accessed"
    }
    
    print("\n🎯 CRITICAL EVENTS AVAILABLE:")
    available_count = 0
    for event_id, description in critical_events.items():
        event_count = recent_events.filter(col("EventID") == event_id).count()
        if event_count > 0:
            print(f"   ✅ {event_id}: {description} ({event_count:,})")
            available_count += 1
        else:
            print(f"   ❌ {event_id}: {description}")
    
    print(f"\n📈 {available_count}/{len(critical_events)} critical event types available")
    
else:
    print("❌ Cannot proceed - SecurityEvent table not accessible")

## Authentication Analysis

Analyze successful and failed logon events to detect authentication threats.

In [8]:
# Authentication events analysis (4624 - Success, 4625 - Failed)
if security_events_available:
    print("🔐 AUTHENTICATION ANALYSIS")
    print("=" * 23)
    
    # Filter to authentication events
    auth_events = recent_events.filter(
        (col("EventID") == 4624) | (col("EventID") == 4625)
    )
    
    auth_count = auth_events.count()
    
    if auth_count > 0:
        print(f"📊 Total authentication events: {auth_count:,}")
        
        # Success vs Failure summary
        auth_summary = auth_events.groupBy("EventID") \
            .agg(count("*").alias("Count"), 
                 countDistinct("Account").alias("UniqueAccounts")) \
            .orderBy("EventID")
        
        auth_summary.show(truncate=False)
        
        # Failed logon analysis
        failed_logons = auth_events.filter(col("EventID") == 4625)
        failed_count = failed_logons.count()
        
        if failed_count > 0:
            print(f"\n❌ FAILED LOGON ANALYSIS ({failed_count:,} events):")
            
            # Top accounts with failed logons
            failed_accounts = failed_logons.groupBy("Account") \
                .agg(count("*").alias("FailedAttempts")) \
                .filter(col("FailedAttempts") > 5) \
                .orderBy(desc("FailedAttempts"))
            
            print("👥 ACCOUNTS WITH MOST FAILURES (>5):")
            failed_accounts.show(5, truncate=False)
            
            # Potential brute force detection
            brute_force = failed_logons.groupBy("Account", "Computer") \
                .agg(count("*").alias("Attempts")) \
                .filter(col("Attempts") > 20) \
                .orderBy(desc("Attempts"))
            
            bf_count = brute_force.count()
            if bf_count > 0:
                print(f"\n🚨 POTENTIAL BRUTE FORCE ({bf_count} cases - >20 attempts):")
                brute_force.show(5, truncate=False)
            else:
                print("\n✅ No brute force patterns detected")
        
        # Successful logon types
        success_logons = auth_events.filter(col("EventID") == 4624)
        if success_logons.count() > 0:
            logon_types = success_logons.filter(col("LogonType").isNotNull()) \
                .groupBy("LogonType") \
                .agg(count("*").alias("Count")) \
                .orderBy(desc("Count"))
            
            print(f"\n✅ LOGON TYPES:")
            logon_types.show(5, truncate=False)
            
    else:
        print("⚠️ No authentication events found")
        
else:
    print("❌ SecurityEvent table not available")

## Process Creation Analysis

Analyze process creation events to identify suspicious process execution.

In [9]:
# Process creation analysis (Event ID 4688)
if security_events_available:
    print("⚡ PROCESS CREATION ANALYSIS")
    print("=" * 27)
    
    # Filter to process creation events
    process_events = recent_events.filter(col("EventID") == 4688)
    process_count = process_events.count()
    
    if process_count > 0:
        print(f"📊 Process creation events: {process_count:,}")
        
        # Extract process information (this may vary based on your data structure)
        # Try to get process name from available fields
        process_analysis = process_events.select(
            "TimeGenerated", "Computer", "Account", "Activity",
            col("EventData").alias("ProcessInfo")  # May need adjustment based on schema
        )
        
        # Show sample process events
        print("\n📋 SAMPLE PROCESS CREATION EVENTS:")
        process_analysis.orderBy(desc("TimeGenerated")).show(5, truncate=False)
        
        # Analyze by computer
        computer_process_stats = process_events.groupBy("Computer") \
            .agg(count("*").alias("ProcessCount"),
                 countDistinct("Account").alias("UniqueAccounts")) \
            .orderBy(desc("ProcessCount"))
        
        print("\n💻 PROCESS CREATION BY COMPUTER:")
        computer_process_stats.show(10, truncate=False)
        
        # Analyze by account
        account_process_stats = process_events.filter(col("Account").isNotNull()) \
            .groupBy("Account") \
            .agg(count("*").alias("ProcessCount"),
                 countDistinct("Computer").alias("UniqueComputers")) \
            .filter(col("ProcessCount") > 50) \
            .orderBy(desc("ProcessCount"))
        
        print("\n👤 HIGH PROCESS CREATION ACCOUNTS (>50 processes):")
        account_process_stats.show(10, truncate=False)
        
        # Time-based analysis
        hourly_processes = process_events \
            .withColumn("Hour", hour(col("TimeGenerated"))) \
            .groupBy("Hour") \
            .agg(count("*").alias("ProcessCount")) \
            .orderBy("Hour")
        
        print("\n🕐 PROCESS CREATION BY HOUR:")
        hourly_processes.show(24, truncate=False)
        
        # Look for potential suspicious patterns
        # High process creation during off-hours (11 PM - 5 AM)
        off_hours_processes = process_events \
            .withColumn("Hour", hour(col("TimeGenerated"))) \
            .filter((col("Hour") >= 23) | (col("Hour") <= 5)) \
            .groupBy("Computer", "Account") \
            .agg(count("*").alias("OffHoursProcesses")) \
            .filter(col("OffHoursProcesses") > 10) \
            .orderBy(desc("OffHoursProcesses"))
        
        off_hours_count = off_hours_processes.count()
        if off_hours_count > 0:
            print(f"\n🌙 OFF-HOURS PROCESS ACTIVITY ({off_hours_count} cases):")
            print("   (>10 processes created during 11 PM - 5 AM)")
            off_hours_processes.show(10, truncate=False)
        else:
            print("\n✅ No significant off-hours process activity detected")
            
    else:
        print("⚠️ No process creation events (4688) found")
        print("📝 This could mean:")
        print("   • Process auditing not enabled")
        print("   • Events not being collected")
        print("   • Different time window needed")
        
else:
    print("❌ SecurityEvent table not available for analysis")

## Account Management Events

Analyze user account creation, deletion, and group membership changes.

In [10]:
# Account management events analysis
if security_events_available:
    print("👥 ACCOUNT MANAGEMENT ANALYSIS")
    print("=" * 29)
    
    # Account management event IDs
    account_mgmt_events = [4720, 4722, 4723, 4724, 4725, 4726, 4728, 4729, 4732, 4733, 4756, 4757]
    
    account_events = recent_events.filter(col("EventID").isin(account_mgmt_events))
    account_count = account_events.count()
    
    if account_count > 0:
        print(f"📊 Account management events: {account_count:,}")
        
        # Event type breakdown
        event_breakdown = account_events.groupBy("EventID", "Activity") \
            .agg(count("*").alias("EventCount")) \
            .orderBy(desc("EventCount"))
        
        print("\n📈 ACCOUNT MANAGEMENT EVENT TYPES:")
        event_breakdown.show(truncate=False)
        
        # Recent account changes
        print("\n📅 RECENT ACCOUNT MANAGEMENT ACTIVITY:")
        account_events.select("TimeGenerated", "EventID", "Activity", "Computer", "Account") \
            .orderBy(desc("TimeGenerated")) \
            .show(10, truncate=False)
        
        # Account creations (4720)
        account_creations = account_events.filter(col("EventID") == 4720)
        creation_count = account_creations.count()
        
        if creation_count > 0:
            print(f"\n➕ ACCOUNT CREATIONS ({creation_count} events):")
            account_creations.select("TimeGenerated", "Computer", "Account") \
                .orderBy(desc("TimeGenerated")) \
                .show(5, truncate=False)
        
        # Account deletions (4726)
        account_deletions = account_events.filter(col("EventID") == 4726)
        deletion_count = account_deletions.count()
        
        if deletion_count > 0:
            print(f"\n➖ ACCOUNT DELETIONS ({deletion_count} events):")
            account_deletions.select("TimeGenerated", "Computer", "Account") \
                .orderBy(desc("TimeGenerated")) \
                .show(5, truncate=False)
        
        # Group membership changes (4728, 4732, 4756)
        group_changes = account_events.filter(
            (col("EventID") == 4728) | (col("EventID") == 4732) | (col("EventID") == 4756)
        )
        group_count = group_changes.count()
        
        if group_count > 0:
            print(f"\n👥 GROUP MEMBERSHIP CHANGES ({group_count} events):")
            group_changes.select("TimeGenerated", "EventID", "Activity", "Computer", "Account") \
                .orderBy(desc("TimeGenerated")) \
                .show(10, truncate=False)
        
        # Activity by computer
        computer_activity = account_events.groupBy("Computer") \
            .agg(count("*").alias("AccountMgmtEvents"),
                 countDistinct("EventID").alias("UniqueEventTypes")) \
            .orderBy(desc("AccountMgmtEvents"))
        
        print("\n💻 ACCOUNT MANAGEMENT ACTIVITY BY COMPUTER:")
        computer_activity.show(10, truncate=False)
        
    else:
        print("⚠️ No account management events found")
        print("📝 This could mean:")
        print("   • No recent account changes")
        print("   • Account management auditing not enabled")
        print("   • Events not being collected")
        
else:
    print("❌ SecurityEvent table not available for analysis")

## Network Access Analysis

Analyze network logons and file share access for lateral movement detection.

In [11]:
# Network and file share access analysis
if security_events_available:
    print("🌐 NETWORK ACCESS ANALYSIS")
    print("=" * 25)
    
    # Network logons (Type 3) from successful logons
    network_logons = recent_events.filter(
        (col("EventID") == 4624) & (col("LogonType") == "3")
    )
    network_count = network_logons.count()
    
    if network_count > 0:
        print(f"📊 Network logons (Type 3): {network_count:,}")
        
        # Network logon patterns
        network_patterns = network_logons.groupBy("Account", "Computer") \
            .agg(count("*").alias("NetworkLogons")) \
            .filter(col("NetworkLogons") > 10) \
            .orderBy(desc("NetworkLogons"))
        
        print("\n🔗 FREQUENT NETWORK LOGONS (>10 logons):")
        network_patterns.show(10, truncate=False)
        
        # Cross-computer access by account
        cross_computer_access = network_logons.groupBy("Account") \
            .agg(countDistinct("Computer").alias("UniqueComputers"),
                 count("*").alias("TotalNetworkLogons")) \
            .filter(col("UniqueComputers") > 3) \
            .orderBy(desc("UniqueComputers"))
        
        cross_count = cross_computer_access.count()
        if cross_count > 0:
            print(f"\n🚨 POTENTIAL LATERAL MOVEMENT ({cross_count} accounts):")
            print("   (Accounts accessing >3 different computers)")
            cross_computer_access.show(10, truncate=False)
        else:
            print("\n✅ No obvious lateral movement patterns detected")
    
    # File share access events (5140)
    share_access = recent_events.filter(col("EventID") == 5140)
    share_count = share_access.count()
    
    if share_count > 0:
        print(f"\n📁 FILE SHARE ACCESS ANALYSIS ({share_count:,} events):")
        
        # Share access by account
        share_by_account = share_access.groupBy("Account") \
            .agg(count("*").alias("ShareAccess"),
                 countDistinct("Computer").alias("UniqueShares")) \
            .filter(col("ShareAccess") > 20) \
            .orderBy(desc("ShareAccess"))
        
        print("👤 HIGH VOLUME SHARE ACCESS (>20 accesses):")
        share_by_account.show(10, truncate=False)
        
        # Recent share access
        print("\n📅 RECENT FILE SHARE ACCESS:")
        share_access.select("TimeGenerated", "Account", "Computer", "Activity") \
            .orderBy(desc("TimeGenerated")) \
            .show(5, truncate=False)
    
    # Remote interactive logons (Type 10 - RDP)
    rdp_logons = recent_events.filter(
        (col("EventID") == 4624) & (col("LogonType") == "10")
    )
    rdp_count = rdp_logons.count()
    
    if rdp_count > 0:
        print(f"\n🖥️ RDP/REMOTE DESKTOP ANALYSIS ({rdp_count:,} events):")
        
        # RDP access patterns
        rdp_patterns = rdp_logons.groupBy("Account", "Computer") \
            .agg(count("*").alias("RDPLogons")) \
            .orderBy(desc("RDPLogons"))
        
        print("🔒 RDP LOGON PATTERNS:")
        rdp_patterns.show(10, truncate=False)
        
        # Multiple RDP destinations by account
        rdp_destinations = rdp_logons.groupBy("Account") \
            .agg(countDistinct("Computer").alias("RDPDestinations"),
                 count("*").alias("TotalRDPLogons")) \
            .filter(col("RDPDestinations") > 2) \
            .orderBy(desc("RDPDestinations"))
        
        rdp_multi_count = rdp_destinations.count()
        if rdp_multi_count > 0:
            print(f"\n🎯 MULTIPLE RDP DESTINATIONS ({rdp_multi_count} accounts):")
            rdp_destinations.show(10, truncate=False)
    
    if network_count == 0 and share_count == 0 and rdp_count == 0:
        print("⚠️ Limited network access events found")
        print("📝 This could mean:")
        print("   • Network access auditing not fully enabled")
        print("   • Limited network activity")
        print("   • Different time window needed")
        
else:
    print("❌ SecurityEvent table not available for analysis")

## Security Risk Assessment

Generate comprehensive security risk assessment based on all findings.

In [ ]:
# Security risk assessment
if security_events_available:
    print("🎯 SECURITY RISK ASSESSMENT")
    print("=" * 27)
    
    # Calculate key metrics
    total_events = recent_events.count()
    failed_logons = recent_events.filter(col("EventID") == 4625).count()
    process_events = recent_events.filter(col("EventID") == 4688).count()
    account_mgmt = recent_events.filter(
        col("EventID").isin([4720, 4726, 4728, 4732])
    ).count()
    unique_computers = recent_events.select("Computer").distinct().count()
    
    print(f"📊 SUMMARY (Last 24 hours):")
    print(f"   Total Events: {total_events:,}")
    print(f"   Failed Logons: {failed_logons:,}")
    print(f"   Process Events: {process_events:,}")
    print(f"   Account Changes: {account_mgmt:,}")
    print(f"   Active Computers: {unique_computers}")
    
    # Simple risk scoring
    risk_score = 0
    risk_factors = []
    
    if failed_logons > 100:
        risk_score += 30
        risk_factors.append(f"High failed logon volume ({failed_logons:,})")
    if account_mgmt > 10:
        risk_score += 25
        risk_factors.append(f"High account activity ({account_mgmt})")
    if process_events > 10000:
        risk_score += 20
        risk_factors.append(f"Very high process activity ({process_events:,})")
    
    # Risk level
    if risk_score >= 50:
        risk_level = "🔴 HIGH"
    elif risk_score >= 25:
        risk_level = "🟡 MEDIUM"
    else:
        risk_level = "🟢 LOW"
    
    print(f"\n🎯 RISK ASSESSMENT:")
    print(f"   Score: {risk_score}/100")
    print(f"   Level: {risk_level}")
    
    if risk_factors:
        print(f"\n⚠️ Key Risk Factors:")
        for factor in risk_factors:
            print(f"   • {factor}")
    
    # Simple hourly chart
    hourly_activity = recent_events \
        .withColumn("Hour", hour(col("TimeGenerated"))) \
        .groupBy("Hour") \
        .agg(count("*").alias("Events")) \
        .orderBy("Hour")
    
    hourly_pd = hourly_activity.toPandas()
    
    if len(hourly_pd) > 0:
        plt.figure(figsize=(10, 4))
        plt.bar(hourly_pd['Hour'], hourly_pd['Events'], color='steelblue', alpha=0.7)
        plt.xlabel('Hour of Day')
        plt.ylabel('Events')
        plt.title('Security Events by Hour')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    
else:
    print("❌ Cannot perform assessment - SecurityEvent data not available")

## Summary

This notebook analyzes Windows Security Events from Microsoft Sentinel Data Lake:

**Key Analysis:**
- Authentication events (successful/failed logons)
- Process creation monitoring
- Account management changes  
- Network access patterns
- Overall security risk assessment

**Key Event IDs:**
- 4624/4625: Logon success/failure
- 4688: Process creation
- 4720/4726: Account create/delete
- 5140: Network share access

**Next Steps:**
- Run analysis regularly for trending
- Set up alerts for high-risk findings
- Customize thresholds for your environment